In [ ]:
import socket
import threading
import os

# Global variables for managing active connections
active_connections = 0
connection_lock = threading.Lock()  # Lock for thread-safe access to active_connections

# Dictionary to map file extensions to their corresponding content types
CONTENT_TYPES = {
    ".html": "text/html",
    ".txt": "text/plain",
    ".jpg": "image/jpeg",
    ".jpeg": "image/jpeg",
    ".png": "image/png"
}

def calculate_timeout():
    """Calculate the timeout duration based on the number of active connections."""
    global active_connections
    if active_connections < 5:
        return 20  # Long timeout if under low load
    elif active_connections < 10:
        return 10  # Medium timeout if load is moderate
    else:
        return 5   # Short timeout if the server is busy

def start_server(host='localhost', port=8080):
    """Start the HTTP server to accept incoming connections."""
    
    # Create a TCP/IP socket
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Bind the socket to the specified host and port
    server_socket.bind((host, port))
    
    # Listen for incoming connections (up to 5 queued connections)
    server_socket.listen(5)  
    print(f"Server started on {host}:{port}")
    
    while True:
        # Accept a new connection from a client
        client_socket, client_address = server_socket.accept()
        print(f"Connection from {client_address}")
        
        # Create a new thread to handle the client connection
        client_thread = threading.Thread(target=handle_client, args=(client_socket,))
        client_thread.start()  # Start the thread

def handle_client(client_socket):
    """Handle incoming client connections and process requests."""
    
    global active_connections
    with connection_lock:  # Ensure thread-safe access to active_connections
        active_connections += 1  # Increment active connection count

    try:
        while True:
            # Set a timeout for the client connection
            client_socket.settimeout(calculate_timeout())
            
            data = b''  # Initialize a variable to store incoming data
            while True:
                # Receive data from the client in chunks
                part = client_socket.recv(1024)
                data += part  # Accumulate the received data
                if b'\r\n\r\n' in data:  # Check for the end of the HTTP headers
                    break
            
            request = data.decode()  # Decode the bytes to a string
            if not request.strip():  # If the request is empty, exit the loop
                break
            
            print(f"Received request:\n{request}")  # Print the received request
            
            # Parse the first line of the request to get the method and path
            request_line = request.splitlines()[0]  
            try:
                method, path, _ = request_line.split()  # Extract method and path
            except ValueError:
                # If request parsing fails, handle it as a bad request
                handle_bad_request(client_socket)
                continue

            # Handle GET and POST requests
            if method == 'GET':
                handle_get(client_socket, path)  # Process a GET request
            elif method == 'POST':
                handle_post(client_socket, path, request)  # Process a POST request
            else:
                handle_unsupported_method(client_socket)  # Handle unsupported HTTP methods

    except socket.timeout:
        print("Connection closed due to timeout")  # Log timeout event
    finally:
        # Decrement the active connection count and close the client socket
        with connection_lock:
            active_connections -= 1
        client_socket.close()  # Close the connection

def handle_get(client_socket, path):
    """Handle HTTP GET requests."""
    
    filepath = path.lstrip("/")  # Remove leading slash from path
    if os.path.exists(filepath):  # Check if the requested file exists
        with open(filepath, 'rb') as f:
            content = f.read()  # Read the file content

        # Determine the content type based on the file extension
        extension = os.path.splitext(filepath)[1]
        content_type = CONTENT_TYPES.get(extension, "application/octet-stream")
        
        # Prepare the HTTP response headers
        response_headers = [
            "HTTP/1.1 200 OK",
            "Connection: keep-alive",
            f"Content-Length: {len(content)}",  # Set content length
            f"Content-Type: {content_type}",  # Set content type
            "\r\n"  # End of headers
        ]
        response = "\r\n".join(response_headers).encode() + content  # Combine headers and content
    else:
        # Prepare a 404 Not Found response if the file does not exist
        response_headers = [
            "HTTP/1.1 404 Not Found",
            "Connection: close",
            "Content-Length: 13",
            "Content-Type: text/plain",
            "\r\n"  # End of headers
        ]
        response = "\r\n".join(response_headers).encode() + b"File Not Found"  # Response body
    
    client_socket.sendall(response)  # Send the response to the client

def handle_post(client_socket, path, request):
    """Handle HTTP POST requests."""
    
    # Split the request into headers and body
    headers, body = request.split("\r\n\r\n", 1)  
    filepath = path.lstrip("/")  # Remove leading slash from path
    
    # Write the body content to the specified file
    with open(filepath, 'wb') as f:
        f.write(body.encode())  # Write the uploaded file content
    
    # Prepare the HTTP response for a successful upload
    response_body = "File Uploaded Successfully"
    response_headers = [
        "HTTP/1.1 200 OK",
        "Connection: keep-alive",
        f"Content-Length: {len(response_body)}",  # Set content length
        "Content-Type: text/plain",  # Set content type
        "\r\n"  # End of headers
    ]
    response = "\r\n".join(response_headers).encode() + response_body.encode()  # Combine headers and body
    client_socket.sendall(response)  # Send the response to the client

def send_response(client_socket, response_text, status="200 OK", content_type="text/plain"):
    """Helper function to send an HTTP response to the client."""
    
    # Prepare HTTP response headers
    response_headers = [
        f"HTTP/1.1 {status}",  # Status line
        "Connection: keep-alive",  # Keep the connection open
        f"Content-Length: {len(response_text)}",  # Set content length
        f"Content-Type: {content_type}",  # Set content type
        "\r\n"  # End of headers
    ]
    response = "\r\n".join(response_headers).encode() + response_text.encode()  # Combine headers and body
    client_socket.sendall(response)  # Send the response to the client

def handle_unsupported_method(client_socket):
    """Handle unsupported HTTP methods by sending a 405 Method Not Allowed response."""
    
    send_response(client_socket, "Method Not Allowed", status="405 Method Not Allowed")

def handle_bad_request(client_socket):
    """Handle bad requests by sending a 400 Bad Request response."""
    
    send_response(client_socket, "Bad Request", status="400 Bad Request")

if __name__ == "__main__":
    # Run the server on localhost and port 8080
    start_server(host='localhost', port=8080)